In [ ]:
import os, base64
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.communication.email import EmailClient
from IPython.display import Image, display
import httpx
from platform import python_version
print(python_version())

load_dotenv()

MAIL_FROM_DOMAIN=os.environ.get('MAIL_FROM_DOMAIN')
MAIL_FROM_USER=os.environ.get('MAIL_FROM_USER')
MAIL_FROM_KEY=os.environ.get('MAIL_FROM_KEY')
MAIL_TO_TEST=os.environ.get('MAIL_TO_TEST')
MAIL_ENDPOINT=os.environ.get('MAIL_ENDPOINT')

AZURE_MAPS_KEY = os.environ.get('AZURE_MAPS_KEY')

In [ ]:
width=600
height=400
zoom=13
fieldop_lat=30.4
fieldop_lon=-97.6
aid1_lat=30.5
aid1_lon=-97.5

pin_instances = [
        f"default|co008000|lcFFFFFF||'OP'{fieldop_lon} {fieldop_lat}",
        f"default|coFFFF00|lc000000||'AID'{aid1_lon} {aid1_lat}"
    ]

url = "https://atlas.microsoft.com/map/static"
params = {
    'subscription-key': AZURE_MAPS_KEY,
    'api-version': '2024-04-01',
    'layer': 'basic',
    'style': 'main',
    'zoom': zoom,
    'center': f'{fieldop_lon},{fieldop_lat}',
    'width': width,
    'height': height,
    'pins': pin_instances,
    'path': f'lcFF1493||{fieldop_lon} {fieldop_lat}|{aid1_lon} {aid1_lat}'
}
response = httpx.get(url, params=params)
image_data = base64.b64encode(response.content).decode()

In [ ]:
with open("diags.png", "rb") as file:
    png_file_bytes_b64 = base64.b64encode(file.read())

inlineAttachments = [
    {
        "name": "diags.png",
        "contentId": "diags_png",
        "contentType": "image/png",
        "contentInBase64":  png_file_bytes_b64.decode()
    }
]

In [ ]:
sender_address = f"{MAIL_FROM_USER}@{MAIL_FROM_DOMAIN}"
print(sender_address)
message = {
    "content": {
        "subject": "Welcome to Azure Communication Services Email",
        "plainText": "This email message is sent from Azure Communication Services Email using the Python SDK.",
        "html": "<html><h1>HTML body inline images:</h1><img src=\"cid:my-inline-attachment-1\" /></html>"
    },
    "recipients": {
        "to": [
            {
                "address": "kris@saveourallies.org",
                "displayName": "Kris DevOps"
            }
        ]
    },
    "senderAddress": f"{sender_address}",
    "attachments": inlineAttachments
}

In [ ]:
connect_string = f"endpoint=https://{MAIL_ENDPOINT}/;accesskey={MAIL_FROM_KEY}"
email_client = EmailClient.from_connection_string(connect_string)

try:
  poller = email_client.begin_send(message)
except Exception as e:
  print(e)

In [ ]:
print(poller.result())